## RAG Approch for summerazation

In [9]:
##Libraries
import os
import chromadb
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
from openai import OpenAI
from sentence_transformers import SentenceTransformer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# Paths
PDF_PATH = r"D:\2CSI-Project\PDFs_papers"  # Folder containing PDFs
CHUNKS_DIR = r"D:\2CSI-Project\Chunks"  # Folder to store chunks
PROGRESS_FILE = r"D:\2CSI-Project\progress.txt"  # File to save the last processed index
VECTORDB_PATH = r"D:\2CSI-Project\VectorDB_Embeddings"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
os.makedirs(CHUNKS_DIR, exist_ok=True)

#### Chroma DB Connection

In [3]:
client = chromadb.PersistentClient(path=VECTORDB_PATH)
collection = client.get_or_create_collection(name='ties_collection_emb', metadata={"hnsw:space": "cosine"})

#### Laod Embedding model

In [4]:
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

#### query & query embedding

In [5]:
query = """cademic community. As an illustration of the scale of the field’s growth, the number of submis-
sions to NeurIPS, a primary conference on ML methods, has quadrupled in six years, going from
1,678submissionsin2014to6,743in2019[ 38].Nevertheless,therearestillplentyofpracticalcon-
siderationsthataffectthemodellearningstage.Inthissection,wediscussissuesconcerningthree
steps within model learning: model selection, training, and hyper-parameter selection.
4.1 Model Selection
In many practical cases the selection of a model is decided by one key characteristic of a model:
complexity.Despiteareassuchasdeeplearningandreinforcementlearninggaininginpopularity
with the research community, in practice simpler models are often chosen. Such models include
shallowneuralnetworkarchitectures,simpleapproachesbasedon PrincipalComponentAnal-
ysis (PCA), decision trees, and random forests.
Simple models can be used as a way to prove the concept of the proposed ML solution and get
the end-to-end setup in place. This approach reduces the time to get a deployed solution, allows
the collection of important feedback, and also helps avoid overcomplicated designs. This was the
case reported by Haldar et al. [39]. In the process of applying machine learning to AirBnB search,
the team started with a complex deep learning model. The team was quickly overwhelmed by its
complexityandendedupconsumingdevelopmentcycles.Afterseveralfaileddeploymentattempts
the neural network architecture was drastically simplified: a single hidden layer NN with 32 fully
connected ReLU activations. Even such a simple model had value, as it allowed the building of a
whole pipeline of deploying ML models"""
query_embedding = embedding_model.encode(query).tolist()



#### Retrieve most similiar chunks

In [6]:
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=["documents", "distances"]
)


In [7]:
## printing chunks
for i in range(len(results["documents"][0])):
    print(f"Chunk: {results['documents'][0][i]}")
    print(f"Similarity Score: {1 - results['distances'][0][i]}")
    print("=" * 80)


Chunk: academic community. As an illustration of the scale of the field’s growth, the number of submis-
sions to NeurIPS, a primary conference on ML methods, has quadrupled in six years, going from
1,678submissionsin2014to6,743in2019[ 38].Nevertheless,therearestillplentyofpracticalcon-
siderationsthataffectthemodellearningstage.Inthissection,wediscussissuesconcerningthree
steps within model learning: model selection, training, and hyper-parameter selection.
4.1 Model Selection
In many practical cases the selection of a model is decided by one key characteristic of a model:
complexity.Despiteareassuchasdeeplearningandreinforcementlearninggaininginpopularity
with the research community, in practice simpler models are often chosen. Such models include
shallowneuralnetworkarchitectures,simpleapproachesbasedon PrincipalComponentAnal-
ysis (PCA), decision trees, and random forests.
Simple models can be used as a way to prove the concept of the proposed ML solution and get
the end-to-end setu